In [202]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns
import scipy
import re

In [460]:
%cd C:\Users\Zak White\AppData\Local\Programs\Python\Python36\Thinkful Datasets
    
df = pd.read_csv('WELLCOME_APCspend2013_forThinkful.csv', encoding = "ISO-8859-1")
#df = pd.read_csv('WELLCOME_APCspend2013_forThinkful.csv')


C:\Users\Zak White\AppData\Local\Programs\Python\Python36\Thinkful Datasets


In [402]:
df.info()
# Looking at the dataframe info we can see many rows missing their PMID
# and another row missing their Journal title.
# The column titles are also messy
# The most glaring problem is there is no consistent data type for any column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2127 entries, 0 to 2126
Data columns (total 5 columns):
PMID/PMCID                                             1928 non-null object
Publisher                                              2127 non-null object
Journal title                                          2126 non-null object
Article title                                          2127 non-null object
COST (£) charged to Wellcome (inc VAT when charged)    2127 non-null object
dtypes: object(5)
memory usage: 83.2+ KB


In [462]:
df.columns = ['PMID/PMCID', 'Publisher', 'Journal_title', 'Article_title',
       'COST(£)']
df.columns

Index(['PMID/PMCID', 'Publisher', 'Journal_title', 'Article_title', 'COST(£)'], dtype='object')

In [424]:
df

,PMID/PMCID,Publisher,Journal_title,Article_title,COST(£)
0,NaN,CUP,psychological medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,j med chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,j med chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,j org chem,Regioselective opening of myo-inositol orthoes...,£685.88
5,PMC3579457,ACS,journal of medicinal chemistry,Comparative Structural and Functional Studies ...,£2392.20
6,PMC3709265,ACS,journal of proteome research,Mapping Proteolytic Processing in the Secretom...,£2367.95
7,23057412 PMC3495574,ACS,mol pharm,Quantitative silencing of EGFP reporter gene b...,£649.33
8,PMCID: PMC3780468,ACS (Amercian Chemical Society) Publications,acs chemical biology,A Novel Allosteric Inhibitor of the Uridine Di...,£1294.59
9,PMCID: PMC3621575,ACS (Amercian Chemical Society) Publications,acs chemical biology,Chemical proteomic analysis reveals the drugab...,£1294.78


In [423]:
#top 5 Journal Title
df['Journal_title']= df['Journal_title'].apply(lambda x : str.lower(str(x)))
df['Journal_title'].value_counts().head()

#Looking at the number of uniques we can see a possible duplicate Article Title
#Possibly multiple issues with the PMID/PMCID column
# It appears there are plenty of articles that are priced similarly which is not an issue.

plos one                           190
journal of biological chemistry     53
neuroimage                          29
plos genetics                       24
plos pathogens                      24
Name: Journal_title, dtype: int64

In [407]:
# NOT NEEDED FOR ASSIGNMENT, BUT GOOD PRACTICE!!!
# Clean IDs
    # PMID = integers of varrying length
    # PMCIDs = PMC+integer
    # remove any row with out a "cleanable" ID
df['PMID/PMCID'].unique()

def condense(x):
    if len(x)<=8 and len(x)>6:
        return x
    else:
        return "NaN"

#Currently clean_ID splits the ID column into a table of elements
    #elements are deliniated from spaces
    #then filtered for digits/ID that are >6 in length, but no more than 8
def clean_ID(x):
    x= pd.Series(str(x).strip().split(" "))
    ##x = list(filter(lambda x: str(x).isdigit(),x))
    ##x = x.apply(lambda x: ''.join(list(filter(str.isdigit, str(x)))))
    x = x.apply(lambda x: condense(''.join(list(filter(str.isdigit, str(x))))))
    return x
ID_table = df['PMID/PMCID'].apply(clean_ID)

ID_table
#condense this table into 1 
#len(ID_table[0][1])
#ID_table

#Regular Expression function filtering for IDs
#### WHHHHAT IN THE ACTUAL FUCK IS THIS SHIT!!!!!
#df['PMID/PMCID'].apply(lambda x :re.findall(r"DMC\?[-+]?\d*\.\d+|\d+",str(x)))
#df['PMID/PMCID'].apply(lambda x :re.findall(r"DMC\?[-+]?\d*\.\d+|\d+",str(x)))

0                               []
1                        [3679557]
2              [23043264, 3506128]
3              [23438330, 3646402]
4              [23438216, 3601604]
5                        [3579457]
6                        [3709265]
7              [23057412, 3495574]
8                        [3780468]
9                        [3621575]
10                       [3739413]
11                       [3530961]
12                       [3624797]
13                       [3413243]
14                       [3694353]
15                       [3572711]
16                      [22610094]
17                       [3586974]
18             [23455506, 3607399]
19             [24015914, 3833349]
20                       [3805332]
21                              []
22                       [3656742]
23                       [3584654]
24             [22971149, 3466778]
25                      [23072351]
26                       [3477822]
27                       [3530960]
28             [2345

In [276]:
from currency_converter import CurrencyConverter

In [270]:
#Clean price data
    # Eliminate currency unit notation
    # convert to numeric
    # If USD
        # Convert to (£)
    # Elimi
# look for "£" & "$" sign on left or right
    #if "$":
        # drop
        #convert to "£"
    #else
        # drop "£"        
def is_sterling(x):
    x.apply(lambda x :re.findall(r"^£(\d+\.?\d)",x))
    if str(x).find('£')
        return str(x).isdigit()
    else
        return str(x).isdigit()

#Cost_table = df['COST(£)'].apply(is_sterling)
#print(*df['COST(£)'].unique(), sep='\n')

In [464]:
# Find all £
# returns an unindexed list
Pounds = df['COST(£)'].apply(lambda x :(re.findall(r"^£(\d+\.?\d+)",str(x))))
Pounds = Pounds.apply(lambda x : 0 if x==[] else float(x[0]))
Pounds

# Find all $
# Convert to £ 1 usd = .76
Dollars = df['COST(£)'].apply(lambda x :(re.findall(r"^(\d+\.?\d+)[$]",str(x))))
Dollars = Dollars.apply(lambda x : 0 if x==[] else float(x[0])*.76)
Dollars

df['COST(£)']= Dollars + Pounds



0          0.00
1       2381.04
2        642.56
3        669.64
4        685.88
5       2392.20
6       2367.95
7        649.33
8       1294.59
9       1294.78
10      1329.69
11      1287.20
12      1324.66
13       665.64
14      1006.72
15       238.08
16       265.67
17      2035.00
18      3108.08
19      1267.76
20      2286.73
21       947.07
22      1186.80
23       642.89
24      1533.29
25      2082.74
26      2314.10
27       759.67
28      1565.26
29       572.74
         ...   
2097    2236.02
2098    2275.02
2099    2373.57
2100     791.19
2101    1551.49
2102    1961.68
2103    1982.35
2104    2239.22
2105    2377.65
2106    2272.15
2107    1900.70
2108    3026.17
2109    1851.15
2110    1812.89
2111    1816.70
2112    2168.05
2113    1409.96
2114    2400.00
2115    1968.63
2116    2015.72
2117    1836.92
2118    1836.92
2119    1375.80
2120    1836.92
2121    2009.65
2122    1334.15
2123    1834.77
2124    1834.77
2125    2374.52
2126    2034.75
Name: COST(£), Length: 2

In [466]:
df['COST(£)'].describe()


#### WHHHHAT IN THE ACTUAL FUCK IS THIS SHIT!!!!!
#df['PMID/PMCID'].apply(lambda x :re.findall(r"DMC\?[-+]?\d*\.\d+|\d+",str(x)))
#df['PMID/PMCID'].apply(lambda x :re.findall(r"DMC\?[-+]?\d*\.\d+|\d+",str(x)))

count      2127.000000
mean      24065.217099
std      146860.994414
min           0.000000
25%        1273.070000
50%        1883.710000
75%        2321.305000
max      999999.000000
Name: COST(£), dtype: float64

In [ ]:
# Dirty list
#money = pd.Series([400, 111, '$20', 57, 'Lots'])

# Here's a lambda function that mirrors the is_a_digit function above.
# Read this print statement carefully and compare to the previous one.
print(money.apply(lambda x: str(x).isdigit()))